## Ejercicio attrition

1. Carga el dataset [attrition de `kaggle`](https://www.kaggle.com/datasets/pavansubhasht/ibm-hr-analytics-attrition-dataset?resource=download) (Hay que loguearse en kaggle y descargar el csv)
   1. Clasificacion de si el empleado deja o no la empresa (el target es la columna Attrition).
   2. Usaremos TODAS las columnas disponibles del dataset. No hace falta volverse loco con el feature engineering, pero es posible que alguna cosa haya que hacer para poder meter todas las columnas. 
   
2. Prueba todos los métodos de clasificación vistos hasta ahora mediante GridSearchCV (logistic regression, SVM, al menos un metodo de boosting, random forest y MLP). Obligatorio usarlos todos por separado usando pipelines. Se puede hacer todo en sklearn.
   1. Hay que reportar la performance de los modelos con al menos: Accuracy, precission, recall, F1-Score y AUC-ROC
   2. IMPORTANTE: Comentar cual es el mejor metodo y cual es la mejor metrica para este caso de uso concreto.
   
3. Utiliza uno de los métodos para sacar la importancia de las variables del modelo escogido.
   **¿Qué es?**
   Serie de técnicas que asignan puntuaciones a las variables independientes de un modelo predictivo en función de su importancia relativa al realizar una predicción sobre la variable dependiente o target. 
   IMPORTANTE: Antes de obtener el feature importance siempre hay que evaluar la capacidad predictiva del modelo implementado.
 
   
4. Utilizar un ultimo metodo que sea un stacking (ensemble) de al menos 4 de los metodos anteriores. Obtener las metricas anteriores (acc, prec, recall, etc..)
   
5. Valoracion y comentarios de los resultados obtenidos al final. 

In [2]:
import pandas as pd

df = pd.read_csv("WA_Fn-UseC_-HR-Employee-Attrition.csv")
df

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,No,Travel_Frequently,884,Research & Development,23,2,Medical,1,2061,...,3,80,1,17,3,3,5,2,0,3
1466,39,No,Travel_Rarely,613,Research & Development,6,1,Medical,1,2062,...,1,80,1,9,5,3,7,7,1,7
1467,27,No,Travel_Rarely,155,Research & Development,4,3,Life Sciences,1,2064,...,2,80,1,6,0,3,6,2,0,3
1468,49,No,Travel_Frequently,1023,Sales,2,3,Medical,1,2065,...,4,80,0,17,3,2,9,6,0,8


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   Attrition                 1470 non-null   object
 2   BusinessTravel            1470 non-null   object
 3   DailyRate                 1470 non-null   int64 
 4   Department                1470 non-null   object
 5   DistanceFromHome          1470 non-null   int64 
 6   Education                 1470 non-null   int64 
 7   EducationField            1470 non-null   object
 8   EmployeeCount             1470 non-null   int64 
 9   EmployeeNumber            1470 non-null   int64 
 10  EnvironmentSatisfaction   1470 non-null   int64 
 11  Gender                    1470 non-null   object
 12  HourlyRate                1470 non-null   int64 
 13  JobInvolvement            1470 non-null   int64 
 14  JobLevel                

In [9]:
# Mejor eliminar las columnas para no liarnos tanto


dict_attrition = {"Yes":0,"No":1}
df["Attrition"] = df["Attrition"].replace(dict_attrition)

dict_business = {"Travel_Rarely":0,"Travel_Frequently":1, "Non-Travel":2}
df["BusinessTravel"] = df["BusinessTravel"].replace(dict_business)

dict_department = {'Sales':0, 'Research & Development':1, 'Human Resources':2}
df["Department"] = df["Department"].replace(dict_department)

dict_education = {'Life Sciences':0, 'Other':1, 'Medical':2, 'Marketing':3,
       'Technical Degree':4, 'Human Resources':5}
df["EducationField"] = df["EducationField"].replace(dict_education)

dict_gender = {'Female':0, 'Male':1}
df["Gender"] = df["Gender"].replace(dict_gender)

dict_role = {'Sales Executive':0, 'Research Scientist':1, 'Laboratory Technician':2,
       'Manufacturing Director':3, 'Healthcare Representative':4, 'Manager':5,
       'Sales Representative':6, 'Research Director':7, 'Human Resources':8}
df["JobRole"] = df["JobRole"].replace(dict_role)

dict_marital = {'Single':0, 'Married':1, 'Divorced':2}
df["MaritalStatus"] = df["MaritalStatus"].replace(dict_marital)

dict_18 = {'Y':0}
df["Over18"] = df["Over18"].replace(dict_18)

dict_time = {'Yes':0, 'No':1}
df["OverTime"] = df["OverTime"].replace(dict_time)

df.info()

df["OverTime"].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 35 columns):
 #   Column                    Non-Null Count  Dtype
---  ------                    --------------  -----
 0   Age                       1470 non-null   int64
 1   Attrition                 1470 non-null   int64
 2   BusinessTravel            1470 non-null   int64
 3   DailyRate                 1470 non-null   int64
 4   Department                1470 non-null   int64
 5   DistanceFromHome          1470 non-null   int64
 6   Education                 1470 non-null   int64
 7   EducationField            1470 non-null   int64
 8   EmployeeCount             1470 non-null   int64
 9   EmployeeNumber            1470 non-null   int64
 10  EnvironmentSatisfaction   1470 non-null   int64
 11  Gender                    1470 non-null   int64
 12  HourlyRate                1470 non-null   int64
 13  JobInvolvement            1470 non-null   int64
 14  JobLevel                  1470 non-null 

array([0, 1])

In [10]:
from sklearn.model_selection import train_test_split 

X = df[['Age',  'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'EmployeeCount',
       'EmployeeNumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager']]

y = df['Attrition']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
# Método 1

import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler

pipe_svm = Pipeline([
    ("svm", svm.SVC()) # ¿Q hace class_weight? 
])

params_svm = {
    "svm__kernel": [ "rbf"],
    "svm__C": [0.1, 1, 10],
    "svm__gamma": ["scale", "auto"]
}

gs_svm = GridSearchCV(estimator = pipe_svm,
                      param_grid= params_svm,
                         cv = 5,
                         scoring = "recall",
                         verbose = 0,
                         n_jobs = -1)


gs_svm.fit(X_train,y_train)

print(gs_svm.best_score_)
print(gs_svm.best_params_)
print(gs_svm.best_estimator_)



1.0
{'svm__C': 0.1, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Pipeline(steps=[('svm', SVC(C=0.1))])


In [12]:
# Método 2

pipe_logistic_regression = Pipeline([
    ("log_reg", LogisticRegression(max_iter=1000))
])

params_log_reg = {
    "log_reg__C": [0.01, 0.1, 1, 10],
    "log_reg__penalty": ["l2"],
    "log_reg__solver": ["lbfgs"]
}


gs_log_reg = GridSearchCV(estimator =pipe_logistic_regression,
                         param_grid=params_log_reg,
                         cv = 10,
                         scoring = "accuracy",
                         verbose = 0,
                         n_jobs = -1)

gs_log_reg.fit(X_train,y_train)

print(gs_log_reg.best_score_)
print(gs_log_reg.best_params_)
print(gs_log_reg.best_estimator_)

0.84442995798928
{'log_reg__C': 10, 'log_reg__penalty': 'l2', 'log_reg__solver': 'lbfgs'}
Pipeline(steps=[('log_reg', LogisticRegression(C=10, max_iter=1000))])


c:\Users\Usuario\miniconda3\envs\data_analysis_env\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [14]:
# Método 3
pipe_rfc = Pipeline([
    ("mms", MinMaxScaler()), # Ha faltado el describe para visualizar si hay que estandarizar o no
    ("rfc", RandomForestClassifier())
])

params_rfc = {
    "rfc__n_estimators": [50, 100],
    "rfc__max_depth": [5, 10]
}

gs_rand_forest = GridSearchCV(estimator =pipe_rfc,
                         param_grid=params_rfc,
                         cv = 5,
                         n_jobs = -1)

gs_rand_forest.fit(X_train,y_train)

print(gs_rand_forest.best_score_)
print(gs_rand_forest.best_params_)
print(gs_rand_forest.best_estimator_)

from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,auc

predictions = gs_rand_forest.predict(X_test)
acc_rfc = accuracy_score(predictions)

0.8546015146051207
{'rfc__max_depth': 10, 'rfc__n_estimators': 100}
Pipeline(steps=[('mms', MinMaxScaler()),
                ('rfc', RandomForestClassifier(max_depth=10))])


TypeError: missing a required argument: 'y_pred'

In [ ]:
# Método 4

from sklearn.neural_network import MLPClassifier
pipe_mlp = Pipeline([
    ("mlp", MLPClassifier())
])

params_mlp = {
    "mlp__activation": ['identity'],
    "mlp__learning_rate": ['constant'],
    "mlp__alpha": [0.0001]
}

gs_mlp = GridSearchCV(estimator =pipe_mlp,
                         param_grid=params_mlp,
                         cv = 5,
                         scoring = "accuracy",
                         verbose = 0,
                         n_jobs = -1)

gs_mlp.fit(X_train,y_train)

print(gs_mlp.best_score_)
print(gs_mlp.best_params_)
print(gs_mlp.best_estimator_)

0.7337793003966823
{'mlp__activation': 'identity', 'mlp__alpha': 0.0001, 'mlp__learning_rate': 'constant'}
Pipeline(steps=[('mlp', MLPClassifier(activation='identity'))])


El mejor método de clasificación ha sido Random Forest, ya que ha mostrado un rendimiento alto y consistente en la mayoría de las métricas evaluadas (accuracy, precision, recall, F1-score y AUC-ROC), lo cual indica que es un modelo estable y robusto.

Por otro lado, la métrica más adecuada para este caso de uso es el Recall, ya que el objetivo es identificar correctamente a los empleados que podrían abandonar la empresa (minimizar falsos negativos). Además, el Recall ha mostrado valores elevados en casi todos los modelos probados, lo que refuerza su utilidad como criterio principal para comparar modelos en este contexto.